In [21]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics.cluster import contingency_matrix
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.cluster import KMeans

data = pd.read_csv("happiness.csv")

main_data = data.drop('Ladder score in Dystopia',
               axis = 1) #after seeing distribution, decided to remove variable as it has one value for all objects

In [ ]:
main_data.head()

In [ ]:
subdata=main_data[["Social support", "Healthy life expectancy", "Freedom to make life choices", "Generosity", "Perceptions of corruption"]]
subdata=(subdata-np.mean(subdata))/(np.max(subdata)-np.min(subdata))
subdata 

In [6]:
best = KMeans(n_clusters=5, n_init=1)
best.fit(subdata)
for i in range(1000):
  model = KMeans(n_clusters=5, n_init=1)
  model.fit(subdata)
  if model.inertia_<best.inertia_:
    best=model
best.inertia_

11.945947024624356

In [7]:
best9 = KMeans(n_clusters=9, n_init=1)
best9.fit(subdata)
for i in range(10000):
  model = KMeans(n_clusters=9, n_init=1)
  model.fit(subdata)
  if model.inertia_<best9.inertia_:
    best9=model
best9.inertia_

In [ ]:
b5=pd.DataFrame(best.cluster_centers_, columns=subdata.columns)*100
#b5.to_excel("b5.xlsx")
b5

In [ ]:
b9=pd.DataFrame(best9.cluster_centers_, columns=subdata.columns)*100
#b9.to_excel("b9.xlsx")
b9

In [10]:
d1=main_data.copy()
d1["c5"]=best.predict(subdata)
d1["c9"]=best9.predict(subdata)


In [ ]:
d1.corr()

In [ ]:
plt.figure(figsize = (10, 8), dpi = 200)
sns.distplot(main_data["Dystopia + residual"], bins=60, kde=False)
plt.grid()
plt.xticks(np.arange(0, 5, 0.2))
plt.savefig("nov1.png");

In [ ]:
plt.figure(figsize = (10, 8), dpi = 200)
sns.distplot(data["Ladder score"], bins=60, kde=False)
plt.grid()
plt.xticks(np.arange(1, 10, 0.4))
plt.savefig("nov2.png");

In [32]:
data["x1"]=main_data["Dystopia + residual"]

In [ ]:
main_data.loc[main_data["Dystopia + residual"]<2, ["x1"]]=1
main_data.loc[main_data["Dystopia + residual"]>=2, ["x1"]]=2
main_data.loc[main_data["Dystopia + residual"]>=3, ["x1"]]=3
main_data

In [ ]:
main_data["x2"]=main_data["Ladder score"]
main_data.loc[main_data["Ladder score"]<4, ["x2"]]=1
main_data.loc[main_data["Ladder score"]>=4, ["x2"]]=2
main_data.loc[main_data["Ladder score"]>=6, ["x2"]]=3
main_data

In [ ]:
plt.figure(figsize = (5, 4), dpi = 200)
plt.title("Pie chart of x1")
plt.pie([len(main_data["x1"][main_data["x1"]==1]), len(main_data["x1"][main_data["x1"]==2]),len(main_data["x1"][main_data["x1"]==3])], labels=["1", "2", "3"], autopct='%1.1f%%')
plt.savefig("x1.png");

In [ ]:
plt.figure(figsize = (5, 4), dpi = 200)
plt.title("Pie chart of x2")
plt.pie([len(main_data["x2"][main_data["x2"]==1]), len(main_data["x2"][main_data["x2"]==2]),len(main_data["x2"][main_data["x2"]==3])], labels=["1", "2", "3"], autopct='%1.1f%%')
plt.savefig("x2.png");

In [ ]:
main_data["part"]=d1["c5"]
main_data

In [44]:
result1=pd.DataFrame(contingency_matrix(main_data["part"], main_data["x1"]), columns=[1,2, 3])
result1["Total"]=[np.sum([result1[i][j] for i in result1]) for j in result1.index]
result1.loc["Total"]=[np.sum([result1[i][j] for j in result1.index]) for i in result1]
result1.to_excel("cont1.xlsx")
result1

,1,2,3,Total
0,7,49,7,63
1,2,18,1,21
2,4,12,9,25
3,6,14,1,21
4,8,10,1,19
Total,27,103,19,149


In [45]:
rel1=result1.copy()
rel1=rel1.divide(149)
rel1.to_excel("rel1.xlsx")
rel1

,1,2,3,Total
0,0.046980,0.328859,0.046980,0.422819
1,0.013423,0.120805,0.006711,0.140940
2,0.026846,0.080537,0.060403,0.167785
3,0.040268,0.093960,0.006711,0.140940
4,0.053691,0.067114,0.006711,0.127517
Total,0.181208,0.691275,0.127517,1.000000


In [47]:
cond1=rel1.copy()
for i in cond1:
  for j in range(cond1.shape[0]-1):
    cond1[i][j]=rel1[i][j]/rel1[i]["Total"]
cond1.to_excel("cond1.xlsx")
cond1

,1,2,3,Total
0,0.259259,0.475728,0.368421,0.422819
1,0.074074,0.174757,0.052632,0.140940
2,0.148148,0.116505,0.473684,0.167785
3,0.222222,0.135922,0.052632,0.140940
4,0.296296,0.097087,0.052632,0.127517
Total,0.181208,0.691275,0.127517,1.000000


In [48]:
m1=np.dot(np.reshape(np.asarray(cond1["Total"][0:-1]), (-1, 1)), np.reshape(np.asarray(cond1.iloc[-1][0:-1]), (1, -1)))
mn1=pd.DataFrame(m1, columns=cond1.columns[:-1])
mn1["Total"]=rel1["Total"][0:-1]
mn1.loc["Total"]=rel1.iloc[-1]
mn1.to_excel("indep1.xlsx")
mn1

,1,2,3,Total
0,0.076618,0.292284,0.053916,0.422819
1,0.025539,0.097428,0.017972,0.140940
2,0.030404,0.115986,0.021395,0.167785
3,0.025539,0.097428,0.017972,0.140940
4,0.023107,0.088149,0.016261,0.127517
Total,0.181208,0.691275,0.127517,1.000000


In [49]:
q1=rel1.copy()
for i in mn1.columns[:-1]:
  for j in range(mn1.shape[0]-1):
    q1[i][j]=q1[i][j]/m1[j][i-1]-1
q1.to_excel("q1.xlsx")
q1

,1,2,3,Total
0,-0.386831,0.125135,-0.128655,0.422819
1,-0.474427,0.239945,-0.626566,0.140940
2,-0.117037,-0.305631,1.823158,0.167785
3,0.576720,-0.035599,-0.626566,0.140940
4,1.323587,-0.238631,-0.587258,0.127517
Total,0.181208,0.691275,0.127517,1.000000


In [50]:
(rel1.drop(["Total"], axis=1).iloc[0:-1]*q1.drop(["Total"], axis=1).iloc[0:-1]).sum().sum()

0.18449654356624984

In [51]:
chi1=(rel1-mn1)**2/mn1
chi1.to_excel("chi1.xlsx")
chi1

,1,2,3,Total
0,0.011465,0.004577,0.000892,0.0
1,0.005748,0.005609,0.007056,0.0
2,0.000416,0.010834,0.071116,0.0
3,0.008495,0.000123,0.007056,0.0
4,0.040481,0.005020,0.005608,0.0
Total,0.000000,0.000000,0.000000,0.0


In [52]:
hi1=chi1.sum().sum()
hi1

0.18449654356624984

In [53]:
stats.chi2.ppf(0.95, (chi1.shape[0]-2)*(chi1.shape[1]-2))/hi1, stats.chi2.ppf(0.99, (chi1.shape[0]-2)*(chi1.shape[1]-2))/hi1

(84.0520519036012, 108.89220275527353)

In [55]:
#for second one
result2=pd.DataFrame(contingency_matrix(main_data["part"], main_data["x2"]), columns=[1,2, 3])
result2["Total"]=[np.sum([result2[i][j] for i in result2]) for j in result2.index]
result2.loc["Total"]=[np.sum([result2[i][j] for j in result2.index]) for i in result2]
result2.to_excel("cont2.xlsx")
result2

,1,2,3,Total
0,0,32,31,63
1,0,1,20,21
2,7,18,0,25
3,2,19,0,21
4,3,15,1,19
Total,12,85,52,149


In [57]:
rel2=result2.copy()
rel2=rel2.divide(149)
rel2.to_excel("rel2.xlsx")
rel2

,1,2,3,Total
0,0.000000,0.214765,0.208054,0.422819
1,0.000000,0.006711,0.134228,0.140940
2,0.046980,0.120805,0.000000,0.167785
3,0.013423,0.127517,0.000000,0.140940
4,0.020134,0.100671,0.006711,0.127517
Total,0.080537,0.570470,0.348993,1.000000


In [58]:
cond2=rel2.copy()
for i in cond2:
  for j in range(cond2.shape[0]-1):
    cond2[i][j]=rel2[i][j]/rel2[i]["Total"]
cond2.to_excel("cond2.xlsx")
cond2

,1,2,3,Total
0,0.000000,0.376471,0.596154,0.422819
1,0.000000,0.011765,0.384615,0.140940
2,0.583333,0.211765,0.000000,0.167785
3,0.166667,0.223529,0.000000,0.140940
4,0.250000,0.176471,0.019231,0.127517
Total,0.080537,0.570470,0.348993,1.000000


In [59]:
m2=np.dot(np.reshape(np.asarray(cond2["Total"][0:-1]), (-1, 1)), np.reshape(np.asarray(cond2.iloc[-1][0:-1]), (1, -1)))
mn2=pd.DataFrame(m2, columns=cond2.columns[:-1])
mn2["Total"]=rel2["Total"][0:-1]
mn2.loc["Total"]=rel2.iloc[-1]
mn2.to_excel("indep2.xlsx")
mn2

,1,2,3,Total
0,0.034053,0.241205,0.147561,0.422819
1,0.011351,0.080402,0.049187,0.140940
2,0.013513,0.095716,0.058556,0.167785
3,0.011351,0.080402,0.049187,0.140940
4,0.010270,0.072744,0.044502,0.127517
Total,0.080537,0.570470,0.348993,1.000000


In [60]:
q2=rel2.copy()
for i in mn2.columns[:-1]:
  for j in range(mn2.shape[0]-1):
    q2[i][j]=q2[i][j]/m2[j][i-1]-1
q2.to_excel("q2.xlsx")
q2

,1,2,3,Total
0,-1.000000,-0.109617,0.409951,0.422819
1,-1.000000,-0.916527,1.728938,0.140940
2,2.476667,0.262118,-1.000000,0.167785
3,0.182540,0.585994,-1.000000,0.140940
4,0.960526,0.383901,-0.849190,0.127517
Total,0.080537,0.570470,0.348993,1.000000


In [61]:
(rel2.drop(["Total"], axis=1).iloc[0:-1]*q2.drop(["Total"], axis=1).iloc[0:-1]).sum().sum()

0.5651518388732013

In [62]:
chi2=(rel2-mn2)**2/mn2
chi2.to_excel("chi2.xlsx")
chi2

,1,2,3,Total
0,0.034053,0.002898,0.024799,0.0
1,0.011351,0.067539,0.147031,0.0
2,0.082887,0.006576,0.058556,0.0
3,0.000378,0.027609,0.049187,0.0
4,0.009475,0.010721,0.032092,0.0
Total,0.000000,0.000000,0.000000,0.0


In [63]:
hi2=chi2.sum().sum()
hi2

0.5651518388732012

In [64]:
stats.chi2.ppf(0.95, (chi2.shape[0]-2)*(chi1.shape[1]-2))/hi2, stats.chi2.ppf(0.99, (chi2.shape[0]-2)*(chi1.shape[1]-2))/hi2

(27.439197732743658, 35.5483847840239)